## 导库

In [1]:
from datasets import load_dataset
import torch
from transformers import BertTokenizer, BertForMaskedLM, Trainer, TrainingArguments

d:\Python\CondaEnvs\Pytorch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 加载数据集

In [2]:
test_file = "./data/MuCGEC_test_filtered.txt"
train_file = "./data/MuCGEC_dev_filtered.txt"

In [3]:
dataset = load_dataset("csv", data_files={
                       "train": train_file, "validation": test_file}, delimiter="\t", column_names=["input", "target"])

Generating train split: 1617 examples [00:00, 103391.76 examples/s]
Generating validation split: 6000 examples [00:00, 236234.49 examples/s]


## 预训练模型

In [5]:
# 加载预训练的中文 BERT 模型和分词器
model_name = "bert-base-chinese"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForMaskedLM.from_pretrained(model_name)

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### 预处理文本数据

In [6]:
def preprocess_function(examples):
    # Ensure inputs are strings and handle potential None values
    input_texts = [
        str(text) if text is not None else "" for text in examples["input"]]
    target_texts = [
        str(text) if text is not None else "" for text in examples["target"]]

    model_inputs = tokenizer(input_texts, max_length=128,
                             truncation=True, padding="max_length")

    # 将目标文本作为标签
    labels = tokenizer(target_texts, max_length=128,
                       truncation=True, padding="max_length")["input_ids"]
    model_inputs["labels"] = labels
    return model_inputs


# 应用预处理
tokenized_datasets = dataset.map(preprocess_function, batched=True)

Map: 100%|██████████| 6000/6000 [00:01<00:00, 3308.42 examples/s]


## 训练

In [7]:
# 定义训练参数
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2
)

# 定义 Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],

)

# 开始训练
trainer.train()

d:\Python\CondaEnvs\Pytorch\lib\site-packages\transformers\training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


[2025-02-25 12:03:19,481] [INFO] [real_accelerator.py:222:get_accelerator] Setting ds_accelerator to cuda (auto detect)


W0225 12:03:20.205000 30352 torch\distributed\elastic\multiprocessing\redirects.py:27] NOTE: Redirects are currently not supported in Windows or MacOs.
d:\Python\CondaEnvs\Pytorch\lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Epoch,Training Loss,Validation Loss
1,No log,4.748908
2,No log,4.897996
3,1.439100,4.912220


TrainOutput(global_step=609, training_loss=1.3478691339101305, metrics={'train_runtime': 174.4459, 'train_samples_per_second': 27.808, 'train_steps_per_second': 3.491, 'total_flos': 319166638737408.0, 'train_loss': 1.3478691339101305, 'epoch': 3.0})

## 预测

In [8]:
def correct_text(text, model, tokenizer, max_length=128):
    # 将模型设置为评估模式
    model.eval()

    # 将输入文本转换为模型输入格式
    inputs = tokenizer(text, return_tensors="pt",
                       max_length=max_length, truncation=True, padding=True)

    # 将输入数据移动到与模型相同的设备
    device = next(model.parameters()).device  # 获取模型所在的设备
    inputs = {key: value.to(device)
              for key, value in inputs.items()}  # 将输入数据移动到相同设备

    # 使用模型预测
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits  # [batch_size, sequence_length, vocab_size]

    # 解码预测结果
    predicted_token_ids = logits.argmax(dim=-1)

    corrected_text = tokenizer.decode(
        predicted_token_ids[0], skip_special_tokens=True)

    return corrected_text

In [9]:
# 示例输入
text = "总得来说，对孩子的教育父母有第一责任的"
corrected_text = correct_text(text, model, tokenizer)
print(f"原始文本: {text}")
print(f"纠错后文本: {corrected_text}")

原始文本: 总得来说，对孩子的教育父母有第一责任的
纠错后文本: 总 的 来 说 ， 对 孩 子 的 教 育 父 母 有 第 一 责 任 任


In [10]:
print_average = 100
with open(test_file, "r", encoding="utf-8") as f:
    lines = f.readlines()
    lines = [line.strip() for line in lines]
    total_lines = len(lines)
    print(f"共有 {total_lines} 行文本")
    print(f"打印 {total_lines // print_average} 轮")

    with open("./data/results.txt", "w", encoding="utf-8") as f:
        for i, line in enumerate(lines):
            if i % print_average == 0:
                print(f"第 {i//total_lines} 轮处理")
            corrected_text = correct_text(
                line, model, tokenizer).replace(" ", '')
            f.write(corrected_text + "\n")

共有 6000 行文本
打印 60 轮
第 0 轮处理
第 0 轮处理
第 0 轮处理
第 0 轮处理
第 0 轮处理
第 0 轮处理
第 0 轮处理
第 0 轮处理
第 0 轮处理
第 0 轮处理
第 0 轮处理
第 0 轮处理
第 0 轮处理
第 0 轮处理
第 0 轮处理
第 0 轮处理
第 0 轮处理
第 0 轮处理
第 0 轮处理
第 0 轮处理
第 0 轮处理
第 0 轮处理
第 0 轮处理
第 0 轮处理
第 0 轮处理
第 0 轮处理
第 0 轮处理
第 0 轮处理
第 0 轮处理
第 0 轮处理
第 0 轮处理
第 0 轮处理
第 0 轮处理
第 0 轮处理
第 0 轮处理
第 0 轮处理
第 0 轮处理
第 0 轮处理
第 0 轮处理
第 0 轮处理
第 0 轮处理
第 0 轮处理
第 0 轮处理
第 0 轮处理
第 0 轮处理
第 0 轮处理
第 0 轮处理
第 0 轮处理
第 0 轮处理
第 0 轮处理
第 0 轮处理
第 0 轮处理
第 0 轮处理
第 0 轮处理
第 0 轮处理
第 0 轮处理
第 0 轮处理
第 0 轮处理
第 0 轮处理
第 0 轮处理
